<a href="https://colab.research.google.com/github/kaixih/EmptyProject/blob/test/JAX_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [ ]:
key = random.PRNGKey(0)
x = random.normal(key, (10,)) # generates random number on device
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


In [ ]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU, block_until_ready is needed since this is async dispatch

14.2 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32) # uses numpy array, which stays on CPU
%timeit jnp.dot(x, x.T).block_until_ready() # slower because the CPU->GPU data transfer

58.2 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x) # explicitly transfers the data to GPU beforehand
%timeit jnp.dot(x, x.T).block_until_ready()

13.7 ms ± 646 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# Program transformation 1: Jit
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

1.16 ms ± 64.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

125 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Program transformation 2: AutoGrad
def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661194 0.10499357]


In [ ]:
def first_finite_differences(f, x):
  eps = 1e-3
  return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps)
                   for v in jnp.eye(len(x))])


print(first_finite_differences(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


In [ ]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0)) # grad and jit can be mixed arbitrarily

-0.0353256


In [ ]:
# Program transformation 3: Vmap
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
  return jnp.dot(mat, v)

In [ ]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
5.56 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
@jit
def batched_apply_matrix(v_batched):
  return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
104 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
@jit
def vmap_batched_apply_matrix(v_batched):
  return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
80 µs ± 9.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
